In [1]:
from default_args import Args
from sqlnet.utils import best_model_name
import torch
from sqlnet.utils import (
    load_dataset,
    load_word_emb,
    best_model_name,
    epoch_acc,
    epoch_exec_acc,
    epoch_train,
    epoch_reinforce_train,
)
from sqlnet.model.seq2sql import Seq2SQL
from sqlnet.model.sqlnet import SQLNet
import datetime
import argparse
import spacy
import numpy as np

In [3]:
args = Args(ca=True)
USE_SMALL = False
GPU = True
BATCH_SIZE = 64
TRAIN_ENTRY = (True, True, True)  # (AGG, SEL, COND)
TRAIN_AGG, TRAIN_SEL, TRAIN_COND = TRAIN_ENTRY
learning_rate = 1e-4 if args.rl else 1e-3

sql_data, table_data, val_sql_data, val_table_data, test_sql_data, \
    test_table_data, TRAIN_DB, DEV_DB, TEST_DB = load_dataset(
        args.dataset, use_small=USE_SMALL
    )

word_emb = load_word_emb(
    "glove/glove.%dB.%dd.txt" % (42, 300),
    load_used=args.train_emb,
    use_small=USE_SMALL,
)

Loading from original dataset
Loading data from data/train_tok.jsonl
Loading data from data/train_tok.tables.jsonl
Loading data from data/dev_tok.jsonl
Loading data from data/dev_tok.tables.jsonl
Loading data from data/test_tok.jsonl
Loading data from data/test_tok.tables.jsonl
Loading word embedding from glove/glove.42B.300d.txt


In [16]:
model = SQLNet(
            word_emb,
            N_word=300,
            use_ca=args.ca,
            gpu=GPU,
            trainable_emb=args.train_emb,
        )

Using fixed embedding
Using column attention on aggregator predicting
Using column attention on selection predicting
Using column attention on where predicting


In [17]:
agg_m, sel_m, cond_m = best_model_name(args, for_load=True)

In [18]:
model.agg_pred.load_state_dict(torch.load(agg_m))
model.sel_pred.load_state_dict(torch.load(sel_m))
model.cond_pred.load_state_dict(torch.load(cond_m))
_ = model.eval()

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [74]:

nlp = spacy.load('en_core_web_sm')

def process_query(q):
    tokens = [c.text.lower() for c in nlp(q)]
    zv = np.zeros(model.embed_layer.N_word, dtype=np.float32)
    X = np.zeros((len(tokens) + 2, model.embed_layer.N_word), dtype=np.float32)
    for i, t in enumerate(tokens):
        X[i + 1] = model.embed_layer.word_emb.get(t, zv.copy())
    var_inp = torch.from_numpy(X)
    if model.gpu:
        var_inp = var_inp.cuda()
    return torch.autograd.Variable(var_inp)

    
text = "What position does the player who played for butler cc (ks) play?"

In [75]:
q = process_query(text)

In [85]:
model.agg_pred(q, np.array(17))

TypeError: iteration over a 0-d array

In [78]:
model.embed_layer.SQL_TOK

['<UNK>', '<END>', 'WHERE', 'AND', 'EQL', 'GT', 'LT', '<BEG>']

In [79]:
q.is_cuda

True